# Direction Rule Validation (Uber)
##### *Algorithms & Data Structures*

A rule looks like this: `A NE B`; this means point `A` is located northeast of point `B`. Another rule looks like this: `A SW C`; this means point `A` is southwest of `C`.

Given a list of rules, check if the sum of the rules validate. For example, the rules below do not validate, since `A` cannot be both north and south of `C`.

In [1]:
from IPython.display import display, HTML
display(HTML('<table><tr><td><img src="img/rules.png"/></td></tr></table>'))

""


### Solution

First, let's break down what it means for a list of rules to be invalid. Consider the following list of rules:
- `A N B`
- `B N A`

The second rule is obviously invalid, since the first node already stated that `B` is North of `A`. We can also see that the following list is equivalent and also invalid:
- `A N B`
- `A S B`

So, we can see that two rules invalidate each other if they relate the same pair of points and are in opposite directions. However, two rules do not invalidate each other if they are in the same direction or the directions are orthogonal to each other:
- `A N B`
- `A E B`

In this case, we see that it is valid for `A` to be both North of `B` and East of `B` at the same time. Let's look at another base case:
- `A N B`
- `C N B`

Here, the relative position of `A` and `C` is ambiguous, other than that they are both North of `B`. Let's now look at an example similar to the original given example:
- `A N B`
- `B N C`
- `C N A`

In this case, we see that `C` cannot be North of `A` because it is implied that `C` is South of `A` by the previous two rules. We could have re-written the first two rules into the following, so that the contradiction is obvious:
- `A N B (original)`
- `B S A`
- `B N C (original)`
- `A N C (through B)`
- `C S B`
- `C S A (through B)`

Then, it is obvious that `C N A` and `C S A` are contradictory. We will perform this expansion and check for contradictions in our algorithm.

Next, we need to figure out how to deal with diagonal cardinal directions (`NE`, `NW`, `SE`, and `SW`). Let's take a look at the case where there are two rules relating the same two points, and the directions are orthogonal (perpendicular) to each other:
- `A N E`
- `A E B`

Notice that these two rules can be simplified into one: `A NE B`. Similarly, we can break down any diagonal direction into the two simple directions (`N`, `E`, `S`, and `W`) that it is composed of.

Now, we can model the relationships between points as a graph. For each included in a given list of rules, there will be a corresponding vertex in the graph. To represent the cardinal directions, each vertex will have a list of edges for each of the four directions. In our solution, we will use directed edges with the convention that an edge `fromVertex DIR toVertex` means `toVertex` is "`DIR` of" `fromVertex`. For example, the rule `A N B` will be parsed into a `N` edge from `B` pointing to `A`, meaning `A` is North of `B`.

When we add a new relationship, we should add a bi-directional edge between the two vertices -- one for the direction in the rule, and one for the opposite. For example, if the rule is `A N B`, we should add an `N` edge from `B` to `A`, and an `S` edge from `A` to `B`.

To add diagonal relationships, we simply parse the two directions into single directions, and treat them as two separate rules.

To validate a rule, we need to check if any existing edges conflict with the new edge(s) we are adding. We compute the relationships between all existing vertices and the new `toVertex`, and cache these within the graph.

Then, we simply check all the neighbors of the `fromVertex`, and return `False` if the neighbor's relationship with `toVertex` is contradictory to the new relationship (i.e. `N` vs `S`, `E` vs `W`).

When we add a new rule, we need to similarly add the relationship to all neighbors of the `fromNode`. For example, say `A` is already North of `B` (and `B` is already South of `A`). If we add the relationship `C S B`, we also add the relationship `C` South of `A` (and `A` North of `C`). If we add the relationship `C` West of `B`, we also add the relationship `C` West of `A` (and `A` East of `C`). However, we do not add a relationship to the neighbors in the same direction as the new relationship, as mentioned in an example above.

In [6]:
N = 0
E = 1
S = 2
W = 3


# A helper class which represents a node in a graph of points/rules as vertices/edges
class Node:
    def __init__(self, char):
        self.edges = [[] for _ in range(4)]
        self.char = char
            

# Calculates the integer representation of the direction opposite of the specified direction
def opposite(direction):
    return (direction + 2) % 4
            

# Checks if a new rule to be added to the graph contradicts an existing rule
def isValid(nodes, fromNode, toNode, direction):
    if toNode in fromNode.edges[opposite(direction)]:
        return False
    
    return True


# Adds the specified rule (of the form `fromNode direction toNode`) to the graph, as well as any additional rules
# that can be reasonably implied by the newly added rule
def addEdges(nodes, fromNode, toNode, direction):
    oppositeDir = opposite(direction)
    
    fromNode.edges[direction].append(toNode)
    toNode.edges[oppositeDir].append(fromNode)
    
    for d in range(4):
        if d == direction:
            continue

        for neighbor in fromNode.edges[d]:
            if neighbor == toNode:
                continue

            neighbor.edges[direction].append(toNode)
            toNode.edges[oppositeDir].append(neighbor)


# The entry point of our algorithm/solution which checks if a given list of rules is valid (i.e., does not contain any
# contradicting rules)
def validate(rules) -> bool:
    nodes = {}
    dirMap = {
        'N': N,
        'E': E,
        'S': S,
        'W': W
    }
    
    for rule in rules:
        temp = rule.split(' ')
        fromVal = temp[2]
        toVal = temp[0]
        
        if not fromVal in nodes:
            nodes[fromVal] = Node(fromVal)
        
        if not toVal in nodes:
            nodes[toVal] = Node(toVal)
            
        fromNode = nodes[fromVal]
        toNode = nodes[toVal]
            
        # Decompose diagonal (two-character) directions into single rules
        for char in temp[1]:
            direction = dirMap[char]
            
            # Check that the new rule does not contradict any existing rules
            if not isValid(nodes, fromNode, toNode, direction):
                return False
            
            addEdges(nodes, fromNode, toNode, direction)
            
    return True
            
            
def test():
    from pprint import pprint as p

    tests = [
        ['A N B', 'C SE B', 'C N A'],
        ['A NW B', 'A N B'],
        ['A N B', 'C N B']
    ]

    for t in tests:
        p(t)
        p(str(validate(t)))
        print()

test()

['A N B', 'C SE B', 'C N A']
'False'

['A NW B', 'A N B']
'True'

['A N B', 'C N B']
'True'



The above solution has the following performance characteristics:
- Time complexity: $O(N * |V|) = O(N^2)$, where N is the number of rules.
- Space complexity: $O(|V| + |E|) = O(|V| + |V|^2) = O(N^2)$, since we are creating a densely-connected graph.